In [ ]:
#get Java
!apt-get install openjdk-8-jdk-headless -qq 

Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 144611 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u265-b01-0ubuntu2~18.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u265-b01-0ubuntu2~18.04) ...
Selecting previously unselected package openjdk-8-jdk-headless:amd64.
Preparing to unpack .../openjdk-8-jdk-headless_8u265-b01-0ubuntu2~18.04_amd64.deb ...
Unpacking openjdk-8-jdk-headless:amd64 (8u265-b01-0ubuntu2~18.04) ...
Setting up openjdk-8-jre-headless:amd64 (8u265-b01-0ubuntu2~18.04) ...
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/orbd to provide /usr/bin/orbd (orbd) in auto mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/servertool to provide /usr/bin/servertool (servertool) in auto mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/tnameserv to provide /usr/bin/tnameserv (tnameserv) in auto mode
Setting up ope

In [ ]:
#get Spark
!wget https://mirrors.estointernet.in/apache/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz 

--2020-10-17 08:57:31--  https://mirrors.estointernet.in/apache/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
Resolving mirrors.estointernet.in (mirrors.estointernet.in)... 43.255.166.254, 2403:8940:3:1::f
Connecting to mirrors.estointernet.in (mirrors.estointernet.in)|43.255.166.254|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 219929956 (210M) [application/octet-stream]
Saving to: ‘spark-3.0.1-bin-hadoop2.7.tgz’

spark-3.0.1-bin-had 100%[===================>] 209.74M  6.00MB/s    in 55s     

2020-10-17 08:58:27 (3.84 MB/s) - ‘spark-3.0.1-bin-hadoop2.7.tgz’ saved [219929956/219929956]



In [ ]:
#extract Spark
!tar xf spark-3.0.1-bin-hadoop2.7.tgz

In [ ]:
#get FindSpark for notebook
!pip install -q findspark

In [ ]:
#set env for Java and Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

In [ ]:
#run findspark
#initialize pyspark
import findspark
findspark.init()

In [ ]:
# import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [ ]:
# create the session
conf = SparkConf()

# create the context
sc = SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [ ]:
#get 20newbytes
!wget http://qwone.com/~jason/20Newsgroups/20news-18828.tar.gz

--2020-10-17 08:58:40--  http://qwone.com/~jason/20Newsgroups/20news-18828.tar.gz
Resolving qwone.com (qwone.com)... 108.20.215.207
Connecting to qwone.com (qwone.com)|108.20.215.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14666916 (14M) [application/x-gzip]
Saving to: ‘20news-18828.tar.gz’

20news-18828.tar.gz 100%[===================>]  13.99M  5.58MB/s    in 2.5s    

2020-10-17 08:58:43 (5.58 MB/s) - ‘20news-18828.tar.gz’ saved [14666916/14666916]



In [ ]:
#extract 20newsbytes
!tar xzf 20news-18828.tar.gz

In [ ]:
#consolidate
!cat 20news-18828/*/* > DataFile.txt

In [ ]:
#create an RDD
data_rdd = sc.textFile('DataFile.txt')

In [ ]:
import fileinput,re
from contextlib import redirect_stdout
import os
import nltk
from nltk.corpus import stopwords
with redirect_stdout(open(os.devnull, "w")): #suppress the output
    nltk.download('stopwords')
sw = stopwords.words('english') 
from nltk.stem import SnowballStemmer
snowball = SnowballStemmer('english')

In [ ]:
def remove_stopwords(text):
  filtered_words = [w for w in text.split()  if ((not w in sw) and (len(w)>1))]
  return " ".join(filtered_words)

In [ ]:
stem = lambda x:snowball.stem(x)

In [ ]:
data_rdd.take(25)

['From: mathew <mathew@mantis.co.uk>',
 'Subject: Alt.Atheism FAQ: Atheist Resources',
 '',
 'Archive-name: atheism/resources',
 'Alt-atheism-archive-name: resources',
 'Last-modified: 11 December 1992',
 'Version: 1.0',
 '',
 '                              Atheist Resources',
 '',
 '                      Addresses of Atheist Organizations',
 '',
 '                                     USA',
 '',
 'FREEDOM FROM RELIGION FOUNDATION',
 '',
 'Darwin fish bumper stickers and assorted other atheist paraphernalia are',
 'available from the Freedom From Religion Foundation in the US.',
 '',
 'Write to:  FFRF, P.O. Box 750, Madison, WI 53701.',
 'Telephone: (608) 256-8900',
 '',
 'EVOLUTION DESIGNS',
 '',
 'Evolution Designs sell the "Darwin fish".  It\'s a fish symbol, like the ones']

In [ ]:
data_rdd.map(lambda x:x.lower()).map(lambda x:x.strip()).map(lambda x :x.replace("\n", " ")).map(lambda x: re.sub("[^a-z]", " ",x)).map(remove_stopwords).map(stem).flatMap(lambda word:word.split(" ")).filter(lambda x : x!='').map(lambda word:(word,1)).reduceByKey(lambda x, y: (x+y)).take(15)

[('mathew', 252),
 ('co', 1253),
 ('subject', 20434),
 ('atheism', 407),
 ('atheist', 581),
 ('resourc', 189),
 ('archive', 408),
 ('name', 1666),
 ('last', 3022),
 ('modified', 167),
 ('decemb', 61),
 ('version', 1310),
 ('addresses', 153),
 ('usa', 830),
 ('religion', 1245)]

In [ ]:
import string
def preprocess(x):
  x = x.lower()
  x = x.strip()
  x = x.replace("\n", " ")
  x = re.sub("[^a-z]", " ",x)
  x = remove_stopwords(x)
  x = ' '.join(list(map(stem,x.split())))
  return x

In [ ]:
import time
start =time.time()
print(data_rdd.map(preprocess).flatMap(lambda word:word.split(" ")).filter(lambda x : x!='').map(lambda word:(word,1)).reduceByKey(lambda x, y: (x+y)).take(10))
print(time.time()-start)

[('mathew', 258), ('manti', 135), ('co', 1226), ('subject', 20657), ('atheism', 537), ('atheist', 982), ('resourc', 765), ('archiv', 764), ('name', 2940), ('last', 3132)]
56.14174199104309


In [ ]:
import time
start =time.time()
print(data_rdd.map(lambda x:x.lower()).map(lambda x:x.strip()).map(lambda x :x.replace("\n", " ")).map(lambda x: re.sub("[^a-z]", " ",x)).flatMap(lambda word:word.split(" ")).map(remove_stopwords).map(stem).filter(lambda x : x!='').map(lambda word:(word,1)).reduceByKey(lambda x, y: (x+y)).take(15))
print(time.time()-start)

[('mathew', 258), ('manti', 135), ('co', 1226), ('subject', 20657), ('atheism', 537), ('atheist', 982), ('resourc', 765), ('archiv', 764), ('name', 2940), ('last', 3132), ('decemb', 193), ('version', 2076), ('usa', 831), ('religion', 1411), ('darwin', 44)]
67.58837246894836


In [ ]:
data_rdd.map(lambda x:x.lower()).map(lambda x:x.strip()).map(lambda x :x.replace("\n", " ")).map(lambda x: re.sub("[^a-z]", " ",x)).flatMap(lambda word:word.split(" ")).map(remove_stopwords).map(stem).filter(lambda x : x!='').map(lambda word:(word,1)).reduceByKey(lambda x, y: (x+y)).take(15)

[('mathew', 258),
 ('manti', 135),
 ('co', 1226),
 ('subject', 20657),
 ('atheism', 537),
 ('atheist', 982),
 ('resourc', 765),
 ('archiv', 764),
 ('name', 2940),
 ('last', 3132),
 ('decemb', 193),
 ('version', 2076),
 ('usa', 831),
 ('religion', 1411),
 ('darwin', 44)]

In [ ]:
data_rdd

DataFile.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0